In [18]:
import requests
import urllib
from bs4 import BeautifulSoup
import string
import re
import pandas as pd


In [19]:
thomasnet_url = 'https://www.thomasnet.com'
base_url = 'https://www.thomasnet.com/nsearch.html?cov=NA&heading=85451102&pg='

#page_letter = [string.ascii_uppercase[i] for i in range(1,2) ]
page_letter = ['D']

In [20]:

def MainPage(mainContent):
    soup = BeautifulSoup(mainContent,'html.parser')
    tags = soup('a')
    #print(tags)
#b = soup.find('div', style = 'border-bottom: 1px solid #cecece; margin-bottom: 18px; padding: 0 0 12px 36px;')
#for i in b.find_all('a'):
    #print(i.get('href'))
    links=[]
    for tag in tags:
        if re.findall("^/browsecompanies\.html\?letter\=[A-Z0-9].+",tag.get('href',None)):
            links.append(re.findall("^/browsecompanies\.html\?letter\=[A-Z0-9].+",tag.get('href',None))[0])

    sub_page_urls = list(set(links))
    return sub_page_urls

In [21]:

# def SubPage(subContent):
#     soup = BeautifulSoup(subContent,'html.parser')
#     tags = soup('a')
#     links=[]
#     for tag in tags:
#         if re.findall("^/profile.+",str(tag.get('href',None))):
#             links.append(re.findall("^/profile.+",tag.get('href',None))[0])
#     manufacturer_profile_urls = list(set(links))
    
#     return manufacturer_profile_urls

def SubPage(subContent):
    soup = BeautifulSoup(subContent,'html.parser')
    tags = soup.find_all('h2', class_ = 'profile-card__title')
    links=[]
    for tag in tags:
        
        links.append(tag.find('a').get('href',None))
    manufacturer_profile_urls = list(set(links))
    
    return manufacturer_profile_urls

In [22]:


def ManufacturerPage(manufacturer_profile_url):
    manufacturerContent = urllib.request.urlopen(thomasnet_url+manufacturer_profile_url).read()
    soup = BeautifulSoup(manufacturerContent,'html.parser')
    
    company_address = []
    company_state = []
    company_phone_tag =[]
    company_tag = soup.find('div', class_='codetail' )
    #company_address = company_tag.find('p', class_ = 'addrline').get_text()
    #company_address = company_tag.find('p', class_ = 'addrline').contents[0][:-2]
    if company_tag:
        
        if company_tag.find('p', class_ = 'addrline'):
            #print(company_tag.find('p', class_ = 'addrline').get_text().split('|')[0])

            if (company_tag.find('p', class_ = 'addrline').get_text().split('|')[0]) == 'map':
                company_address = ''
                company_state = ''
            else :
                company_address = company_tag.find('p', class_ = 'addrline').get_text().split('|')[0]
                #print(company_address, type(company_address))
                company_state = company_address.split(', ')[1].split(' ')[0]

    
        supplier_data = company_tag.find('h1')
        supplier_name = supplier_data.get_text()

        company_tag = soup.find('div', class_='codetail' )
        company_phone_tag = company_tag.find('p', class_ = 'phoneline')

    supplier_number = []
    supplier_url = []
    if company_phone_tag:
        supplier_number = company_phone_tag.find('span').get_text()
        if company_phone_tag.find('a', class_ = "primary"):
            supplier_url = company_phone_tag.find('a', class_ = "primary").get('href')      
    
    #for web_address in company_phone_tag('a'):
     #   supplier_url = []
      #  if web_address.get('data-advertiser_link'):
       #     supplier_url = web_address.get('data-advertiser_link')
    #supplier_url = company_phone_tag.find('a', class_ = "primary").get('href')
    company_description_tag = soup.find('div', id= 'copro_about')
    supplier_description = []
    
    if company_description_tag:
        supplier_description = company_description_tag.get_text()
    
    #---------------Employees---------------------------------
    
    employees_tag = soup.find(class_ = 'bdcol2 match-height').find_all(class_ = 'bizdetail')
    employees_number = ''
    for tag in employees_tag:
        if tag.find(class_='label'):
            if tag.find(class_='label').text == 'No of Employees:':
                employees_number = str(tag.find('li').get_text())
        
    #------------- Capabilities----------------------------------------
    Capabilities_url = soup.find(class_="submenu").find_all('a')[1].get('href')
    CapabilitiesContent = urllib.request.urlopen(thomasnet_url+Capabilities_url).read()
    soup1 = BeautifulSoup(CapabilitiesContent,'html.parser')
    
    Capabilities_list= soup1.find_all(class_= 'expand show')
    
    supplier_capabilities_detailed = []
    supplier_capabilities_overall = []
    Machining = []
    Milling = []
    Turning = []
    for element in Capabilities_list:  
        #print('1')
        supplier_capabilities_detailed.append(element.text)
        supplier_capabilities_overall.append(element.a.text)
        #print(element.get_text()+'\n')
    #------------- Machining Milling Turning-----------------------------------------

    #for element1 in Capabilities_list:
        if re.findall("^Machining.*",element.a.text ):
            for element2 in element.find_all('li'):
                Machining.append(element2.text)
        if re.findall("^Milling.*",element.a.text ):
            for element2 in element.find_all('li'):
                Milling.append(element2.text)
        if re.findall("^Turning.*",element.a.text ):
            for element2 in element.find_all('li'):
                Turning.append(element2.text)
    
    #print(Capabilities)
    
    return company_address, supplier_name, supplier_number, supplier_url, supplier_description,supplier_capabilities_detailed, employees_number, Machining, Milling, Turning, supplier_capabilities_overall,company_state



In [23]:
manufacturer_url = '/profile/30848258/calt-dynamics.html?cid=30848258&cov=NA&heading=45340403&searchpos=3062&what=Machining'


In [24]:
manufacturer_profile_urls = pd.read_csv('manufacturer_url.csv')

In [25]:


def main():
    #sub_page_url = []
    #for letter in page_letter:
     #   mainContent = urllib.request.urlopen(base_url+letter).read()

      #  sub_page_url += (MainPage(mainContent))

    manufacturer_profile_urls = []
    for sub_link in range(1,94):
        subContent = urllib.request.urlopen(base_url+str(sub_link)).read()
        manufacturer_profile_urls += SubPage(subContent)

      #  global Supplier
       # Supplier = pd.DataFrame() 
    print(len(manufacturer_profile_urls))
    myfile = open('manufacturer_url_Casting.csv', 'w')
    mylist = map(lambda x:x+'\n', manufacturer_profile_urls)
    myfile.writelines(mylist)
    myfile.close()
    #for manufacturer_url in manufacturer_profile_urls['Profile_URL']:
    #    print(manufacturer_url)
     #   Manufacturer_details= ManufacturerPage(manufacturer_url)
      #  supplier = {}
       # supplier['Address'] = Manufacturer_details[0]
        #supplier['Name']= Manufacturer_details[1]
        #supplier['Number'] = Manufacturer_details[2]
        #supplier['URL'] = Manufacturer_details[3]
        #supplier['Description'] = Manufacturer_details[4][1:]
        #supplier['Capabilities_Detailed'] = Manufacturer_details[5]
        #supplier['Employees'] = Manufacturer_details[6]
        #supplier['Machining_capabilities'] = Manufacturer_details[7]
        #supplier['Milling_capabilities'] = Manufacturer_details[8]
        #supplier['Turning_capabilities'] = Manufacturer_details[9]
        #supplier['Capabilities_Overall'] = Manufacturer_details[10]
        #supplier['Manufacturer_state'] = Manufacturer_details[11]

        #Supplier.append(supplier)
        #Supplier = Supplier.append(supplier, ignore_index = True)
        #Supplier= Supplier[['Name','Address','Manufacturer_state','Number','URL','Employees','Description','Machining_capabilities','Milling_capabilities','Turning_capabilities','Capabilities_Overall','Capabilities_Detailed']]
        
        #print(Supplier)

        #Supplier.to_csv('Supplier-CNC.csv')
        







In [26]:
%%time
main()

2314
Wall time: 3min 30s


In [185]:
Supplier

,Name,Address,Manufacturer_state,Number,URL,Employees,Description,Machining_capabilities,Milling_capabilities,Turning_capabilities,Capabilities_Overall,Capabilities_Detailed
0,"Ferguson Engineering Co., Inc.",,,833-979-1906,https://www.fergusonhydraulics.com/,10-49,Company Description by Thomasnet\nRemanufactur...,"[Machining: CNC, Machining, Machining: 5-Axis,...","[Milling, Milling: CNC, Milling: Metal, Millin...","[Lathe Work, Turning, Turning: CNC, Turning: M...","[Assembly Machinery, Automotive Parts, Bending...","[Assembly MachineryMachinery: Custom, Automate..."
1,Napoleon Machine,"Napoleon, OH 43545",OH,888-903-1554,https://www.napoleonmachine.com/,10-49,Company Description by Thomasnet\nJob shop ser...,"[Machining: CNC, Machining, Machining: CNC, La...","[Milling, Milling: CNC, Milling: Horizontal, M...","[Turning, Turning: CNC, Turning: Metal, Turnin...","[Boring Services, Broaching Services, Centers,...",[Boring ServicesBoring ServicesBoring Services...
2,"Fedtech, Inc.","Saint Paul, MN 55112",MN,888-784-4600,https://www.fedtech.com/,50-99,Company Description by Thomasnet\nISO 9001:201...,"[Machining, Machining: CNC, Brass Machining, C...","[Milling, Milling: CNC, Milling: Metal, Millin...","[Lathe Work, Screw Machine Products & Turned P...","[Abrasive Compounds / Media, Acrylic, Aerospac...",[Abrasive Compounds / MediaAbrasives: Waterjet...
3,LeFiell Manufacturing Co.,"Santa Fe Springs, CA 90670",CA,800-451-5971,http://www.lefiell.com,100-199,Company Description by Thomasnet\nISO 9002 cer...,"[Machining: CNC, Aircraft & Aerospace, Machining]",[],[],"[Assemblies, Aircraft Parts, Aluminum, Cases, ...","[AssembliesAssemblies: Aircraft & Aerospace, A..."
4,Michigan Mechanical Services,"Taylor, MI 48180-4054",MI,866-669-9730,http://www.michiganmechanical.com/,10-49,Company Description by Thomasnet\nMechanical s...,"[Machining, Machining: On-Site, Field & Mobile...","[Milling, Milling: Metal]","[Lathe Work, Lathe Work: Large Swing]","[Analytical Services, Automation Systems, Bala...",[Analytical ServicesVibration Analysis Service...
...,...,...,...,...,...,...,...,...,...,...,...,...
352,"Ace Laser Technologies, Inc.","Elk River, MN 55330",MN,763-274-0175,http://www.ace-laser.com/,1-9,Company Description by Thomasnet\nLaser cuttin...,"[Machining, Machining: Aluminum, Machining: CN...",[Milling: CNC],[],"[Additive Manufacturing, Cutting Services, Dis...",[Additive ManufacturingRapid Prototyping Servi...
353,Mica-Tron Products Corp.,"Holbrook, MA 02343",MA,800-258-6422,http://www.mica-tron.com/,1-9,Company Description by Thomasnet\nISO 9001:201...,"[Machining: CNC, Aircraft & Aerospace, Ceramic...","[Milling, Milling: CNC, Milling: Plastics, Mil...","[Turning, Turning: CNC, Turning: Plastic, Turn...","[Alumina (Aluminum Oxide), Absorbers, Acetal, ...",[Alumina (Aluminum Oxide)Ceramics: MachinableC...
354,Orange County Industrial Plastics (OCIP),"Anaheim, CA 92807",CA,888-339-8302,https://www.ocip.com/,10-49,Company Description by Thomasnet\nCustom plast...,"[Machining, Machining: Acetal, Machining: Acry...",[],[],"[Acetal, Acrylic, Acrylonitrile Butadiene Styr...",[AcetalAcetalPlastic MaterialsPlastic Material...
355,"Utitec, Inc.","Watertown, CT 06795",CT,855-430-9664,https://utitec.com/,50-99,Company Description by Thomasnet\nCustom manuf...,"[Machining: Medical, Brass Machining, EDM, Mac...","[Milling, Milling: CNC, Milling: Metal, Millin...","[Lathe Work, Screw Machine Products & Turned P...","[Assemblies, COVID-19 Response, Cans, Cases, E...",[AssembliesAssembliesAssemblies: ElectronicAss...
